# Proyek Akhir Robotika


## Anggota:
- Anel
- Adnan
- Rio
- Tegar
- Alif

## Planning

### Identifikasi Masalah


1. Masalah Utama (Tracking Bola):
    - Bagaimana mengembangkan robot yang mampu mendeteksi dan melacak bola secara akurat dalam berbagai kondisi pencahayaan dan lingkungan?
    - Tantangan: Deteksi bola harus real-time dan konsisten, sementara robot harus bisa menggerakkan dirinya ke arah bola dengan responsif menggunakan motor DC.
2. Optimasi Tracking:Masalah: 
    - Bagaimana memastikan robot bisa melacak bola secara sempurna dan menghindari kesalahan deteksi atau pelacakan, seperti kehilangan target atau kesalahan gerakan robot saat mengikuti bola.
    -Tantangan: Perlu penyesuaian algoritma deteksi dan gerakan yang dinamis untuk berbagai kecepatan dan arah pergerakan bola.
3. Fitur Tambahan (Menendang dan Mengenal Gawang)(Optional, jika 2 poin diatas sudah diatasi):
    - Masalah: Setelah pelacakan bola sempurna, bagaimana menambahkan fitur agar robot bisa mengenali dan menendang bola ke arah target, serta mengenali gawang sebagai tujuan akhir.
    - Tantangan: Implementasi model tambahan atau pengenalan objek lebih kompleks, seperti deteksi gawang, serta kontrol motorik yang lebih halus untuk menendang bola dengan kekuatan dan akurasi yang tepat.

### Modeling

1. Modeling (Deteksi Warna Awal)
Kami akan menggunakan kamera dan HSV color space untuk mendeteksi warna bola secara real-time. Motor DC akan menggerakkan robot sesuai dengan posisi bola yang terdeteksi, dengan algoritma yang menggunakan informasi posisi X dan Y bola.

2. Modeling Lanjutan (YOLO)
Ketika deteksi warna sudah berjalan dengan baik, kami akan beralih ke metode YOLO untuk mendeteksi bola secara lebih presisi dan dalam kondisi pencahayaan yang lebih kompleks. YOLO akan dilatih menggunakan dataset dengan label bounding box yang bisa kami peroleh dari Kaggle atau melalui pelabelan manual. Setelah model YOLO dilatih, kami akan mengimplementasikan untuk mendeteksi bola dalam video secara real-time. Posisi bounding box bola yang terdeteksi akan digunakan untuk menggerakkan robot secara akurat mengikuti bola.


### Data Gathering

1. Data Gathering
Untuk tahap awal, kami dapat merekam video bola dengan warna yang mencolok dan kontras tinggi di berbagai kondisi pencahayaan. Setelah video diambil, kami akan mengekstrak beberapa frame untuk melatih dan menguji model. Frame yang diambil akan dilabeli warna bola untuk membantu sistem dalam mengenali targetnya.

2. Dataset Kaggle
Selain itu, kami juga dapat memanfaatkan dataset yang sudah ada di Kaggle untuk mempercepat proses. Contoh dataset yang relevan bisa ditemukan di Kaggle, seperti dataset bola sepak yang sudah dilabeli. Dataset dari Kaggle akan sangat membantu dalam fase pelatihan, baik untuk deteksi berbasis warna maupun YOLO.

### Preprocessing Data

1. Preprocessing Data
Kami akan mengonversi citra dari format RGB ke HSV agar lebih mudah dalam mendeteksi warna spesifik dari bola. Selanjutnya, kami terapkan thresholding pada warna bola untuk membuat area bola lebih jelas dan terisolasi dari latar belakang.
2. Preprocessing Lanjutan
Jika menggunakan dataset gambar, tentunya preprocessing akan terdiri dari resizing, pengahpusan noise. Tak kalah penting akan dilakukan labeling dengan bounding box untuk memberikan info posisi bola sebelum di latih

### Ekstraksi Fitur

1. Ekstraksi Fitur
Setelah deteksi warna dilakukan, kami akan mengekstrak posisi bola dengan menghitung centroid dari area bola yang terdeteksi. Posisi X dan Y dari centroid ini akan digunakan untuk mengontrol arah gerakan robot secara otomatis, menggerakkannya ke arah bola.

## Excecution

### Progress Pertama (Kode Mengenal Bola dengan Contour Warna)

In [2]:
import cv2
import imutils
import numpy as np

ball_hsv_values = {
    "Blue": {  # Rentang HSV untuk bola biru
        "lower": (89, 100, 100),  
        "upper": (109, 255, 255)
    },
    "Green": {  # Rentang HSV untuk bola hijau
        "lower": (61, 100, 100),
        "upper": (81, 255, 255) 
    },
    "Yellow": {  # Rentang HSV untuk bola kuning
        "lower": (20, 100, 100),
        "upper": (30, 255, 255)
    },
    "Orange": {  # Rentang HSV untuk bola oranye gelap
        "lower": (10, 100, 100),   # Rentang lebih rendah untuk hue
        "upper": (15, 255, 255)   # Rentang lebih tinggi untuk hue
    }
}






In [1]:
# Kelas BallTracker 
class BallTracker:
    MIN_BALL_RADIUS = 10  
    MAX_BALL_RADIUS = 60
    FRAME_WIDTH = 500 

    # Konstruktor
    def __init__(self, color: str) -> None:
        assert color in ball_hsv_values 
        self.color = color
        self.lower_bound = ball_hsv_values[color]['lower']  
        self.upper_bound = ball_hsv_values[color]['upper'] 
        self.camera = cv2.VideoCapture(0)  

    # Fungsi untuk mengambil frame video dan mengubahnya ke format HSV
    def get_video_frame(self):
        ret, frame_in_bgr = self.camera.read()  
        if not ret:
            raise Exception('Unable to read from the camera')  
        frame_in_bgr = imutils.resize(frame_in_bgr, width=self.FRAME_WIDTH) 
        frame_in_hsv = cv2.cvtColor(frame_in_bgr, cv2.COLOR_BGR2HSV)  
        return frame_in_bgr, frame_in_hsv  

    # Fungsi untuk membuat mask dari frame berdasarkan rentang warna HSV
    def get_mask(self, frame_in_hsv):
        mask = cv2.inRange(frame_in_hsv, self.lower_bound, self.upper_bound)  
        mask = cv2.erode(mask, None, iterations=2) 
        mask = cv2.dilate(mask, None, iterations=2) 
        return mask

    # Fungsi untuk menemukan kontur terbesar dari mask (yang diasumsikan sebagai bola)
    def get_contour(self, mask):
        contours = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours = contours[0]  
        if len(contours) > 0:
            return max(contours, key=cv2.contourArea)  # Mengembalikan kontur terbesar (asumsi sebagai bola)
        return None  

    # Fungsi utama
    def track_ball(self, display_on_screen=True):
        while True:
            frame_in_bgr, frame_in_hsv = self.get_video_frame()
            mask = self.get_mask(frame_in_hsv) 
            contour = self.get_contour(mask)

            # Menggambar sistem koordinat Kartesius di tengah frame
            height, width, _ = frame_in_bgr.shape
            cv2.line(frame_in_bgr, (width // 2, 0), (width // 2, height), (255, 0, 0), 1)
            cv2.line(frame_in_bgr, (0, height // 2), (width, height // 2), (255, 0, 0), 1)
            
            if contour is not None:
            # Mencari lingkaran yang melingkupi kontur dan mengambil posisinya
                ((x, y), radius) = cv2.minEnclosingCircle(contour)
                if radius > self.MIN_BALL_RADIUS:
                    x, y, radius = int(x), int(y), int(radius)  # Konversi ke integer
                    cv2.circle(frame_in_bgr, (x, y), radius, (0, 0, 255), 2)  # Gambar lingkaran di sekitar bola

                    # Periksa apakah bola cukup besar dan berada di tengah
                    if radius > 100 and (width // 2 - 50 < x < width // 2 + 50):
                        command = "Tendang"
                    else:
                        command = ""  # Kosongkan jika tidak ada perintah tendang

                    # Tentukan arah gerakan robot berdasarkan posisi bola
                    if x < width // 2 - 50:  # Bola berada di kiri
                        movement = "Turn Left"
                    elif x > width // 2 + 50:  # Bola berada di kanan
                        movement = "Turn Right"
                    else:  # Bola berada di tengah
                        movement = "Move Forward"

                    # Tampilkan instruksi gerakan pada frame
                    cv2.putText(frame_in_bgr, movement, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    # Tampilkan perintah tendang jika ada
                    if command:
                        cv2.putText(frame_in_bgr, command, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            if display_on_screen:
                cv2.imshow("Ball Tracking", frame_in_bgr)
            if cv2.waitKey(30) & 0xFF == ord("q"):
                break

        self.camera.release()
        cv2.destroyAllWindows()


In [3]:
tracker = BallTracker("Yellow") 
tracker.track_ball()

NameError: name 'ball_hsv_values' is not defined

In [ ]:
import serial
import time
# from ball_tracker import BallTracker  # Asumsikan file BallTracker terpisah

# Inisialisasi serial Bluetooth
bluetooth = serial.Serial('COM18', 9600)  # Ganti COM5 sesuai port Bluetooth
time.sleep(2)  # Tunggu koneksi stabil

print("Connected to Arduino!")

# Inisialisasi tracker
tracker = BallTracker(color="Yellow")

def send_command(command):
    bluetooth.write(command.encode())  # Kirim perintah sebagai byte
    print(f"Sent: {command}")

try:
    while True:
        _, frame_in_hsv = tracker.get_video_frame()
        mask = tracker.get_mask(frame_in_hsv)
        contour = tracker.get_contour(mask)

        if contour is not None:
            (x, y), radius = cv2.minEnclosingCircle(contour)
            x, y, radius = int(x), int(y), int(radius)
            frame_width = tracker.FRAME_WIDTH

            # Logika arah gerakan
            if radius > 10:  # Hanya bergerak jika bola terdeteksi
                if x < frame_width // 2 - 50:
                    send_command('L')  # Belok kiri
                elif x > frame_width // 2 + 50:
                    send_command('R')  # Belok kanan
                else:
                    send_command('F')  # Maju
            else:
                send_command('S')  # Berhenti jika radius kecil

        else:
            send_command('S')  # Berhenti jika tidak ada bola

        # Tekan 'q' untuk keluar
        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

except KeyboardInterrupt:
    print("Terminated by user")

finally:
    bluetooth.close()
    tracker.camera.release()
    cv2.destroyAllWindows()


Connected to Arduino!


yg ini

In [7]:
import cv2
import imutils
import serial
import time

class BallTracker:
    MIN_BALL_RADIUS = 10  
    FRAME_WIDTH = 500 

    def __init__(self, color: str, bluetooth_port: str, baud_rate: int = 38400) -> None:
        assert color in ball_hsv_values, "Warna tidak ditemukan dalam konfigurasi HSV"
        self.color = color
        self.lower_bound = ball_hsv_values[color]['lower']  
        self.upper_bound = ball_hsv_values[color]['upper'] 
        self.camera = cv2.VideoCapture(1)  # Inisialisasi kamera
        self.bluetooth = serial.Serial(bluetooth_port, baud_rate)  # Inisialisasi komunikasi Bluetooth
        time.sleep(2)  # Tunggu koneksi Bluetooth stabil
        print("Connected to Bluetooth!")

    def get_video_frame(self):
        ret, frame_in_bgr = self.camera.read()  
        if not ret:
            raise Exception('Unable to read from the camera')  
        frame_in_bgr = imutils.resize(frame_in_bgr, width=self.FRAME_WIDTH) 
        frame_in_hsv = cv2.cvtColor(frame_in_bgr, cv2.COLOR_BGR2HSV)  
        return frame_in_bgr, frame_in_hsv  

    def get_mask(self, frame_in_hsv):
        mask = cv2.inRange(frame_in_hsv, self.lower_bound, self.upper_bound)  
        mask = cv2.erode(mask, None, iterations=2) 
        mask = cv2.dilate(mask, None, iterations=2) 
        return mask

    def get_contour(self, mask):
        contours = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours = contours[0]  
        if len(contours) > 0:
            return max(contours, key=cv2.contourArea)  # Mengembalikan kontur terbesar (asumsi sebagai bola)
        return None  

    def send_command(self, command: str):
        """Mengirimkan perintah ke Arduino melalui Bluetooth."""
        self.bluetooth.write(command.encode())  # Mengirimkan string sebagai byte
        print(f"Sent: {command}")  # Debugging log

    def track_ball(self):
        try:
            while True:
                frame_in_bgr, frame_in_hsv = self.get_video_frame()
                mask = self.get_mask(frame_in_hsv)
                contour = self.get_contour(mask)

                if contour is not None:
                    (x, y), radius = cv2.minEnclosingCircle(contour)
                    x, y, radius = int(x), int(y), int(radius)

                    if radius > self.MIN_BALL_RADIUS:  # Jika radius cukup besar
                        if x < self.FRAME_WIDTH // 2 - 50:  # Bola di kiri
                            self.send_command('L')  # Belok kiri
                        elif x > self.FRAME_WIDTH // 2 + 50:  # Bola di kanan
                            self.send_command('R')  # Belok kanan
                        else:  # Bola di tengah
                            self.send_command('F')  # Maju
                    else:
                        self.send_command('S')  # Radius terlalu kecil, berhenti

                else:
                    self.send_command('S')  # Tidak ada bola, berhenti

                # Tekan 'q' untuk keluar
                if cv2.waitKey(30) & 0xFF == ord('q'):
                    break

        except KeyboardInterrupt:
            print("Tracking terminated by user")

        finally:
            self.camera.release()
            self.bluetooth.close()
            cv2.destroyAllWindows()

# Contoh konfigurasi HSV untuk warna bola
ball_hsv_values = {
    "Yellow": {"lower": (20, 100, 100), "upper": (30, 255, 255)},
}

# Inisialisasi dan jalankan tracker
tracker = BallTracker(color="Yellow", bluetooth_port="COM19")  # Ganti COM15 sesuai port Bluetooth Anda
tracker.track_ball()


Connected to Bluetooth!
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: R
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: R
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S
Sent: S


## coba

In [3]:
tracker = BallTracker("Green") 
tracker.track_ball()

Kode di atas bertujuan untuk melacak bola menggunakan deteksi warna berbasis HSV dan menggerakkan robot berdasarkan posisi bola. Jika kontur bola terdeteksi, sistem menghitung lingkaran yang mengelilingi bola untuk menentukan posisi X dan Y serta radius bola. Berdasarkan posisi bola di frame, robot akan mengambil tindakan seperti "Turn Left", "Turn Right", atau "Move Forward". Jika bola cukup besar dan berada di tengah frame, robot akan mengeluarkan perintah "Tendang". Perintah ini serta instruksi gerakan ditampilkan pada video yang diproses, memungkinkan pengendalian robot secara real-time.

### Progress Keuda (Kode Mengenal Bola dengan Contour Warna + pyesrial)

In [ ]:
import cv2
import imutils
import numpy as np
import serial
import time

ball_hsv_values = {
    "Blue": {  # Rentang HSV untuk bola biru
        "lower": (89, 100, 100),  
        "upper": (109, 255, 255)
    },
    "Green": {  # Rentang HSV untuk bola hijau
        "lower": (61, 100, 100),
        "upper": (81, 255, 255) 
    },
    "Yellow": {  # Rentang HSV untuk bola kuning
        "lower": (20, 100, 100),
        "upper": (30, 255, 255)
    },
    "Orange": {  # Rentang HSV untuk bola oranye gelap
        "lower": (5, 100, 100),   # Rentang lebih rendah untuk hue
        "upper": (15, 255, 255)   # Rentang lebih tinggi untuk hue
    }
}
class BallTracker:
    MIN_BALL_RADIUS = 10  
    MAX_BALL_RADIUS = 60
    FRAME_WIDTH = 500 

    def __init__(self, color: str, serial_port: str, baud_rate: int = 9600) -> None:
        assert color in ball_hsv_values
        self.color = color
        self.lower_bound = ball_hsv_values[color]['lower']
        self.upper_bound = ball_hsv_values[color]['upper']
        self.camera = cv2.VideoCapture(1)
        self.serial_connection = serial.Serial(serial_port, baud_rate)
        time.sleep(2)

    def get_video_frame(self):
        ret, frame_in_bgr = self.camera.read()
        if not ret:
            raise Exception('Unable to read from the camera')
        frame_in_bgr = imutils.resize(frame_in_bgr, width=self.FRAME_WIDTH)
        frame_in_hsv = cv2.cvtColor(frame_in_bgr, cv2.COLOR_BGR2HSV)
        return frame_in_bgr, frame_in_hsv

    def get_mask(self, frame_in_hsv):
        mask = cv2.inRange(frame_in_hsv, self.lower_bound, self.upper_bound)
        mask = cv2.erode(mask, None, iterations=2)
        mask = cv2.dilate(mask, None, iterations=2)
        return mask

    def get_contour(self, mask):
        contours = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours = contours[0] if len(contours) == 2 else contours[1]
        if contours:
            return max(contours, key=cv2.contourArea)
        return None

    def track_ball(self, display_on_screen=True):
        while True:
            frame_in_bgr, frame_in_hsv = self.get_video_frame()
            mask = self.get_mask(frame_in_hsv)
            contour = self.get_contour(mask)

            height, width, _ = frame_in_bgr.shape
            cv2.line(frame_in_bgr, (width // 2, 0), (width // 2, height), (255, 0, 0), 1)
            cv2.line(frame_in_bgr, (0, height // 2), (width, height // 2), (255, 0, 0), 1)
            
            command = ""
            if contour is not None:
                ((x, y), radius) = cv2.minEnclosingCircle(contour)
                if radius > self.MIN_BALL_RADIUS:
                    x, y, radius = int(x), int(y), int(radius)
                    cv2.circle(frame_in_bgr, (x, y), radius, (0, 0, 255), 2)

                    if radius > 100 and (width // 2 - 50 < x < width // 2 + 50):
                        command = "T"  
                    if x < width // 2 - 50:
                        movement = "L"  
                    elif x > width // 2 + 50: 
                        movement = "R" 
                    else:
                        movement = "F"  

                    cv2.putText(frame_in_bgr, movement, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    if command == "T":
                        cv2.putText(frame_in_bgr, "Tendang", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

                    self.send_command_to_arduino(movement)
                    if command == "T":
                        self.send_command_to_arduino(command)
                else:
                    self.send_command_to_arduino("S")
            else:
                self.send_command_to_arduino("S")  

            if display_on_screen:
                cv2.imshow("Ball Tracking", frame_in_bgr)
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break

        self.camera.release()
        cv2.destroyAllWindows()

    def send_command_to_arduino(self, command: str) -> None:
        self.serial_connection.write(command.encode())
        time.sleep(0.1)


In [ ]:
tracker = BallTracker("Yellow", serial_port='COM15')  
tracker.track_ball()  


Progress 3

In [1]:
import cv2
import imutils
import numpy as np
import serial
import time

ball_hsv_values = {
    "Blue": {  # Rentang HSV untuk bola biru
        "lower": (89, 100, 100),
        "upper": (109, 255, 255)
    },
    "Green": {  # Rentang HSV untuk bola hijau
        "lower": (61, 100, 100),
        "upper": (81, 255, 255)
    },
    "Yellow": {  # Rentang HSV untuk bola kuning
        "lower": (20, 100, 100),
        "upper": (30, 255, 255)
    },
    "Orange": {  # Rentang HSV untuk bola oranye gelap
        "lower": (5, 100, 100),
        "upper": (15, 255, 255)
    },
    "Orange_Pingpong": {  # Rentang HSV untuk bola pingpong oranye terang
        "lower": (10, 150, 150),  # H: 10, S: 150, V: 150 untuk warna oranye terang
        "upper": (25, 255, 255)   # H: 25, S: 255, V: 255 untuk intensitas penuh
    }
}

class BallTracker:
    MIN_BALL_RADIUS = 10
    MAX_BALL_RADIUS = 60
    FRAME_WIDTH = 500
    no_ball_timeout = 3  # Durasi tanpa deteksi bola sebelum berputar (detik)

    def __init__(self, color: str, serial_port: str, baud_rate: int = 9600) -> None:
        assert color in ball_hsv_values
        self.color = color
        self.lower_bound = ball_hsv_values[color]['lower']
        self.upper_bound = ball_hsv_values[color]['upper']
        self.camera = cv2.VideoCapture(0)
        self.serial_connection = serial.Serial(serial_port, baud_rate)
        self.last_ball_detected_time = time.time()  # Waktu terakhir bola terdeteksi
        time.sleep(2)

    def get_video_frame(self):
        ret, frame_in_bgr = self.camera.read()
        if not ret:
            raise Exception('Unable to read from the camera')
        frame_in_bgr = imutils.resize(frame_in_bgr, width=self.FRAME_WIDTH)
        frame_in_hsv = cv2.cvtColor(frame_in_bgr, cv2.COLOR_BGR2HSV)
        return frame_in_bgr, frame_in_hsv

    def get_mask(self, frame_in_hsv):
        mask = cv2.inRange(frame_in_hsv, self.lower_bound, self.upper_bound)
        mask = cv2.erode(mask, None, iterations=2)
        mask = cv2.dilate(mask, None, iterations=2)
        return mask

    def get_contour(self, mask):
        contours = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours = contours[0] if len(contours) == 2 else contours[1]
        if contours:
            return max(contours, key=cv2.contourArea)
        return None

    def track_ball(self, display_on_screen=True):
        while True:
            frame_in_bgr, frame_in_hsv = self.get_video_frame()
            mask = self.get_mask(frame_in_hsv)
            contour = self.get_contour(mask)

            height, width, _ = frame_in_bgr.shape
            cv2.line(frame_in_bgr, (width // 2, 0), (width // 2, height), (255, 0, 0), 1)
            cv2.line(frame_in_bgr, (0, height // 2), (width, height // 2), (255, 0, 0), 1)

            command = ""
            if contour is not None:
                self.last_ball_detected_time = time.time()  # Update waktu terakhir bola terdeteksi
                ((x, y), radius) = cv2.minEnclosingCircle(contour)
                if radius > self.MIN_BALL_RADIUS:
                    x, y, radius = int(x), int(y), int(radius)
                    cv2.circle(frame_in_bgr, (x, y), radius, (0, 0, 255), 2)

                    if radius > 100 and (width // 2 - 50 < x < width // 2 + 50):
                        command = "T"
                    if x < width // 2 - 50:
                        movement = "L"
                    elif x > width // 2 + 50:
                        movement = "R"
                    else:
                        movement = "F"

                    cv2.putText(frame_in_bgr, movement, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    if command == "T":
                        cv2.putText(frame_in_bgr, "Tendang", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

                    self.send_command_to_arduino(movement)
                    if command == "T":
                        self.send_command_to_arduino(command)
                else:
                    self.send_command_to_arduino("S")
            else:
                # Periksa apakah durasi tanpa deteksi bola sudah melebihi batas
                if time.time() - self.last_ball_detected_time > self.no_ball_timeout:
                    self.send_command_to_arduino("L")  # Berputar jika tidak ada bola
                else:
                    self.send_command_to_arduino("S")  # Berhenti jika bola hilang sementara

            if display_on_screen:
                cv2.imshow("Ball Tracking", frame_in_bgr)
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break

        self.camera.release()
        cv2.destroyAllWindows()

    def send_command_to_arduino(self, command: str) -> None:
        self.serial_connection.write(command.encode())
        time.sleep(0.1)


In [ ]:
tracker = BallTracker("Yellow", serial_port='COM15')  
tracker.track_ball()  


In [2]:
import cv2
import imutils
import numpy as np
import serial
import time

ball_hsv_values = {
    "Blue": {  # Rentang HSV untuk bola biru
        "lower": (89, 100, 100),
        "upper": (109, 255, 255)
    },
    "Green": {  # Rentang HSV untuk bola hijau
        "lower": (61, 100, 100),
        "upper": (81, 255, 255)
    },
    "Yellow": {  # Rentang HSV untuk bola kuning
        "lower": (20, 100, 100),
        "upper": (30, 255, 255)
    },
    "Orange": {  # Rentang HSV untuk bola oranye gelap
        "lower": (5, 100, 100),
        "upper": (15, 255, 255)
    },
    "Orange_Pingpong": {  # Rentang HSV untuk bola pingpong oranye terang
        "lower": (10, 150, 150),  # H: 10, S: 150, V: 150 untuk warna oranye terang
        "upper": (25, 255, 255)   # H: 25, S: 255, V: 255 untuk intensitas penuh
    }
}

class BallTracker:
    MIN_BALL_RADIUS = 10
    MAX_BALL_RADIUS = 60
    FRAME_WIDTH = 500
    no_ball_timeout = 3  # Durasi tanpa deteksi bola sebelum berputar (detik)

    def __init__(self, color: str, serial_port: str, baud_rate: int = 9600) -> None:
        assert color in ball_hsv_values
        self.color = color
        self.lower_bound = ball_hsv_values[color]['lower']
        self.upper_bound = ball_hsv_values[color]['upper']
        self.camera = cv2.VideoCapture(0)
        self.serial_connection = serial.Serial(serial_port, baud_rate)  # Initialize Bluetooth serial
        self.last_ball_detected_time = time.time()  # Waktu terakhir bola terdeteksi
        time.sleep(2)

    def get_video_frame(self):
        ret, frame_in_bgr = self.camera.read()
        if not ret:
            raise Exception('Unable to read from the camera')
        frame_in_bgr = imutils.resize(frame_in_bgr, width=self.FRAME_WIDTH)
        frame_in_hsv = cv2.cvtColor(frame_in_bgr, cv2.COLOR_BGR2HSV)
        return frame_in_bgr, frame_in_hsv

    def get_mask(self, frame_in_hsv):
        mask = cv2.inRange(frame_in_hsv, self.lower_bound, self.upper_bound)
        mask = cv2.erode(mask, None, iterations=2)
        mask = cv2.dilate(mask, None, iterations=2)
        return mask

    def get_contour(self, mask):
        contours = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours = contours[0] if len(contours) == 2 else contours[1]
        if contours:
            return max(contours, key=cv2.contourArea)
        return None

    def track_ball(self, display_on_screen=True):
        while True:
            frame_in_bgr, frame_in_hsv = self.get_video_frame()
            mask = self.get_mask(frame_in_hsv)
            contour = self.get_contour(mask)

            height, width, _ = frame_in_bgr.shape
            cv2.line(frame_in_bgr, (width // 2, 0), (width // 2, height), (255, 0, 0), 1)
            cv2.line(frame_in_bgr, (0, height // 2), (width, height // 2), (255, 0, 0), 1)

            command = ""
            if contour is not None:
                self.last_ball_detected_time = time.time()  # Update waktu terakhir bola terdeteksi
                ((x, y), radius) = cv2.minEnclosingCircle(contour)
                if radius > self.MIN_BALL_RADIUS:
                    x, y, radius = int(x), int(y), int(radius)
                    cv2.circle(frame_in_bgr, (x, y), radius, (0, 0, 255), 2)

                    if radius > 100 and (width // 2 - 50 < x < width // 2 + 50):
                        command = "T"
                    if x < width // 2 - 50:
                        movement = "L"
                    elif x > width // 2 + 50:
                        movement = "R"
                    else:
                        movement = "F"

                    cv2.putText(frame_in_bgr, movement, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    if command == "T":
                        cv2.putText(frame_in_bgr, "Tendang", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

                    self.send_command_to_arduino(movement)
                    if command == "T":
                        self.send_command_to_arduino(command)
                else:
                    self.send_command_to_arduino("S")
            else:
                # Periksa apakah durasi tanpa deteksi bola sudah melebihi batas
                if time.time() - self.last_ball_detected_time > self.no_ball_timeout:
                    self.send_command_to_arduino("L")  # Berputar jika tidak ada bola
                else:
                    self.send_command_to_arduino("S")  # Berhenti jika bola hilang sementara

            if display_on_screen:
                cv2.imshow("Ball Tracking", frame_in_bgr)
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break

        self.camera.release()
        cv2.destroyAllWindows()

    def send_command_to_arduino(self, command: str) -> None:
        self.serial_connection.write(command.encode())  # Send command via Bluetooth
        time.sleep(0.1)


In [ ]:
tracker = BallTracker("Yellow", serial_port='COM18')  
tracker.track_ball()  


In [3]:
import cv2
import imutils
import numpy as np
import serial
import time

# Rentang HSV untuk warna bola
ball_hsv_values = {
    "Blue": {  # Rentang HSV untuk bola biru
        "lower": (89, 100, 100),
        "upper": (109, 255, 255)
    },
    "Green": {  # Rentang HSV untuk bola hijau
        "lower": (61, 100, 100),
        "upper": (81, 255, 255)
    },
    "Yellow": {  # Rentang HSV untuk bola kuning
        "lower": (20, 100, 100),
        "upper": (30, 255, 255)
    },
    "Orange": {  # Rentang HSV untuk bola oranye gelap
        "lower": (5, 100, 100),
        "upper": (15, 255, 255)
    },
    "Orange_Pingpong": {  # Rentang HSV untuk bola pingpong oranye terang
        "lower": (10, 150, 150),  # H: 10, S: 150, V: 150 untuk warna oranye terang
        "upper": (25, 255, 255)   # H: 25, S: 255, V: 255 untuk intensitas penuh
    }
}

class BallTracker:
    MIN_BALL_RADIUS = 10
    MAX_BALL_RADIUS = 60
    FRAME_WIDTH = 500
    no_ball_timeout = 3  # Durasi tanpa deteksi bola sebelum berputar (detik)

    def __init__(self, color: str, serial_port: str, baud_rate: int = 9600) -> None:
        assert color in ball_hsv_values
        self.color = color
        self.lower_bound = ball_hsv_values[color]['lower']
        self.upper_bound = ball_hsv_values[color]['upper']
        self.camera = cv2.VideoCapture(0)
        
        # Periksa apakah kamera terdeteksi
        if not self.camera.isOpened():
            print("Error: Camera not found.")
            return
        
        self.serial_connection = serial.Serial(serial_port, baud_rate)
        self.last_ball_detected_time = time.time()  # Waktu terakhir bola terdeteksi
        time.sleep(2)

    def get_video_frame(self):
        ret, frame_in_bgr = self.camera.read()
        if not ret:
            raise Exception('Unable to read from the camera')
        frame_in_bgr = imutils.resize(frame_in_bgr, width=self.FRAME_WIDTH)
        frame_in_hsv = cv2.cvtColor(frame_in_bgr, cv2.COLOR_BGR2HSV)
        return frame_in_bgr, frame_in_hsv

    def get_mask(self, frame_in_hsv):
        mask = cv2.inRange(frame_in_hsv, self.lower_bound, self.upper_bound)
        mask = cv2.erode(mask, None, iterations=2)
        mask = cv2.dilate(mask, None, iterations=2)
        return mask

    def get_contour(self, mask):
        contours = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours = contours[0] if len(contours) == 2 else contours[1]
        if contours:
            return max(contours, key=cv2.contourArea)
        return None

    def track_ball(self, display_on_screen=True):
        while True:
            frame_in_bgr, frame_in_hsv = self.get_video_frame()
            mask = self.get_mask(frame_in_hsv)
            contour = self.get_contour(mask)

            height, width, _ = frame_in_bgr.shape
            cv2.line(frame_in_bgr, (width // 2, 0), (width // 2, height), (255, 0, 0), 1)
            cv2.line(frame_in_bgr, (0, height // 2), (width, height // 2), (255, 0, 0), 1)

            command = ""
            if contour is not None:
                self.last_ball_detected_time = time.time()  # Update waktu terakhir bola terdeteksi
                ((x, y), radius) = cv2.minEnclosingCircle(contour)
                if radius > self.MIN_BALL_RADIUS:
                    x, y, radius = int(x), int(y), int(radius)
                    cv2.circle(frame_in_bgr, (x, y), radius, (0, 0, 255), 2)

                    if radius > 100 and (width // 2 - 50 < x < width // 2 + 50):
                        command = "T"
                    if x < width // 2 - 50:
                        movement = "L"
                    elif x > width // 2 + 50:
                        movement = "R"
                    else:
                        movement = "F"

                    cv2.putText(frame_in_bgr, movement, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    if command == "T":
                        cv2.putText(frame_in_bgr, "Tendang", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

                    self.send_command_to_arduino(movement)
                    if command == "T":
                        self.send_command_to_arduino(command)
                else:
                    self.send_command_to_arduino("S")
            else:
                # Periksa apakah durasi tanpa deteksi bola sudah melebihi batas
                if time.time() - self.last_ball_detected_time > self.no_ball_timeout:
                    self.send_command_to_arduino("L")  # Berputar jika tidak ada bola
                else:
                    self.send_command_to_arduino("S")  # Berhenti jika bola hilang sementara

            if display_on_screen:
                cv2.imshow("Ball Tracking", frame_in_bgr)
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break

        self.camera.release()
        cv2.destroyAllWindows()

    def send_command_to_arduino(self, command: str) -> None:
        self.serial_connection.write(command.encode())
        time.sleep(0.1)




In [ ]:
# Uji coba dengan bola berwarna biru, port serial Bluetooth, dan baud rate 9600
if __name__ == "__main__":
    tracker = BallTracker("Yellow", "COM15")  # Ganti dengan port serial yang sesuai
    tracker.track_ball()

In [1]:
import serial
import time

# Tentukan port serial dan baud rate
bluetooth = serial.Serial("COMx", 9600)  # Ganti COMx dengan port yang sesuai

time.sleep(2)  # Tunggu beberapa detik agar koneksi Bluetooth stabil

def send_command(command):
    bluetooth.write(command.encode())  # Kirim perintah ke Arduino

# Contoh pengendalian motor dengan Bluetooth
send_command('F')  # Maju
time.sleep(2)  # Motor maju selama 2 detik
send_command('S')  # Berhenti
time.sleep(1)
send_command('B')  # Mundur
time.sleep(2)  # Motor mundur selama 2 detik
send_command('S')  # Berhenti

bluetooth.close()  # Tutup koneksi Bluetooth


SerialException: could not open port 'COMx': FileNotFoundError(2, 'The system cannot find the file specified.', None, 2)